In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.feature import local_binary_pattern
# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(r'D:\MTECH\PROJECT\DM\SECRET\CASCADE\haarcascade_frontalface_default.xml')

# List of emotions
emotions = ['anger','disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Number of orthogonal vectors to keep
num_vectors = 40

# Dictionary to store feature vectors for each emotion
feature_vectors = {emotion: [] for emotion in emotions}
# List to store images and labels
images = []
labels = []

# Path to the dataset directory
dataset_dir = r'D:\MTECH\PROJECT\CV\FINAL - ORIGINAL\DATASET\AffectNet\data_relabeled_balanced_1x\train'

# Dictionary to store feature vectors for each emotion
feature_vectors = {emotion: [] for emotion in emotions}

# Iterate over the images in the dataset
for emotion in emotions:
    emotion_dir = os.path.join(dataset_dir, emotion)
    image_files = os.listdir(emotion_dir)

    for image_file in image_files:
        image_path = os.path.join(emotion_dir, image_file)

        # Load the input image
        image = cv2.imread(image_path)

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale image
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.5, minNeighbors=5, minSize=(30, 30))

        # Iterate over the detected faces
        for (x, y, w, h) in faces:
            # Extract the face region
            face = image[y:y+h, x:x+w]

            # Resize the face image to 40x40 pixels
            resized_face = cv2.resize(face, (30, 30), interpolation=cv2.INTER_LINEAR)

            # Apply bilateral filtering to remove noise
            denoised_face = cv2.bilateralFilter(resized_face, 9, 80, 90)

            # Convert the denoised face image to grayscale
            gray_face = cv2.cvtColor(denoised_face, cv2.COLOR_BGR2GRAY)

            # Calculate LBP (Local Binary Patterns) values
            lbp = local_binary_pattern(gray_face, 8, 1, method='uniform')

            # Calculate the b-bin histogram
            histogram, _ = np.histogram(lbp.ravel(), bins=128, range=(0, 128), density=True)

            # Append the histogram to the feature vectors list for the current emotion
            feature_vectors[emotion].append(histogram)


In [7]:

# Encode emotion labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


# Convert feature vectors and labels to NumPy arrays
feature_vectors = np.array(feature_vectors)
labels = np.array(labels)

# Apply PCA to reduce the dimensionality
scaler = StandardScaler()
scaled_feature_vectors = scaler.fit_transform(feature_vectors)

pca = PCA(n_components=num_vectors)
reduced_feature_vectors = pca.fit_transform(scaled_feature_vectors)

# Split the dataset into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    reduced_feature_vectors, labels, test_size=0.2, random_state=42
)

# Define the CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(emotions), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32)

# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Predict emotions for the testing set
predicted_labels = model.predict(test_images)
predicted_labels = np.argmax(predicted_labels, axis=1)

# Convert the predicted labels back to original emotion labels
predicted_emotions = label_encoder.inverse_transform(predicted_labels)

# Generate classification report
unique_labels = label_encoder.inverse_transform(np.unique(test_labels))
report = classification_report(test_labels, predicted_labels, target_names=unique_labels)
print("Classification Report:\n", report)


NameError: name 'StandardScaler' is not defined